In [3]:
!pip install flickrapi

In [61]:
# import packages
import flickrapi
import getpass
import time
from datetime import datetime
import json
import pandas as pd

In [2]:
API_KEY = getpass.getpass("Enter your API Key")

Enter your API Key ································


In [3]:
API_SECRET = getpass.getpass("Enter your API Secret")

Enter your API Secret ················


In [4]:
# create the API class
flickr = flickrapi.FlickrAPI(API_KEY, API_SECRET, cache=True)

In [67]:
# retriving a set of photos within 1 km of the uw madiosn picnic point (43.08962391993143, -89.41554886446347)
photos = flickr.walk(lat=43.0896, lon=-89.4155, radius=1, extras='url_c', max_upload_date='2023-02-20')
# iterate photos in the photo set
photos_id_list = []
for i, photo_url in enumerate(photos):
    if i > 50: # only collect the first 50 photos
        break
    # get the photo url
    url = photo_url.get('url_c')
    url_id = photo_url.get('id')
    if url is not None:
        photos_id_list.append(url_id)
#print(photos_id_list)

In [25]:
#photos_id_list = []
#photos_id_list.append(url_id)
#print(photos_id_list)

In [55]:
for i in photos_id_list:
    #print(i)
    photo_info = flickr.photos.getInfo(photo_id=i, format='json')
    time.sleep(3)  # wait for 3 seconds before making the next request
    #break
    print("photo information: ", photo_info)
    print('type is: ',type(photo_info))
    print("decode: ", type(photo_info.decode()))
    photo_info_d = photo_info.decode()
     with open('photo_json.json','w') as file:
        file.write(photo_info_d)
    photo_info_l = json.loads(photo_info_d)
    print("information load: ", photo_info_d)

photo information:  b'{"photo":{"id":"52611393971","secret":"ce668c525a","server":"65535","farm":66,"dateuploaded":"1673044631","isfavorite":0,"license":"0","safety_level":"0","rotation":0,"originalsecret":"ff50ab3362","originalformat":"jpg","owner":{"nsid":"76136201@N05","username":"American Life League","realname":"American Life League","location":"","iconserver":"5788","iconfarm":6,"path_alias":"americanlifeleague","gift":{"gift_eligible":true,"eligible_durations":["year","month","week"],"new_flow":true}},"title":{"_content":"PLWisconsin_13"},"description":{"_content":"Pro-Life Wisconsin"},"visibility":{"ispublic":1,"isfriend":0,"isfamily":0},"dates":{"posted":"1673044631","taken":"2000-05-13 00:00:00","takengranularity":0,"takenunknown":"0","lastupdate":"1673045400"},"views":"17","editability":{"cancomment":0,"canaddmeta":0},"publiceditability":{"cancomment":1,"canaddmeta":0},"usage":{"candownload":1,"canblog":0,"canprint":0,"canshare":0},"comments":{"_content":"0"},"notes":{"note"

In [36]:
# retrieve the information (json format) of the photo with id 49307295822
#photo_info = flickr.photos.getInfo(photo_id=52611814230, format='json')
#print("photo information: ", photo_info)

In [68]:
photo_list = []

for i in photos_id_list:
    photo_info = flickr.photos.getInfo(photo_id=i, format='json')
    photo_info_d = photo_info.decode()
    photo_info_l = json.loads(photo_info_d)
    user_info = flickr.people.getInfo(api_key = API_KEY, user_id = photo_info_l["photo"]["owner"]["nsid"], format = 'json')
    user_info_d = user_info.decode()
    user_info_l = json.loads(user_info_d)
    
    photo_id = photo_info_l ['photo']['id']
    latitude = photo_info_l ['photo'] ['location'] ['latitude']
    longitude = photo_info_l ['photo'] ['location'] ['longitude']
    taken_time = photo_info_l ['photo'] ['dates'] ['taken']
    url = photo_info_l ['photo'] ['urls'] ['url'][0] ['_content']
    user_id = photo_info_l ['photo'] ['owner'] ['nsid']
    username = photo_info_l ['photo'] ['owner'] ['username']
    realname = photo_info_l ['photo'] ['owner'] ['realname']
    registered_location = photo_info_l ['photo']['owner']['location']
    photos_count = user_info_l ['person'] ['photos'] ['count'] ['_content']
    
    tags = photo_info_l ['photo'] ['tags'] ['tag']
    if tags:
        user_tags = [tag['_content'] for tag in tags]
        #print(user_tags)
    else:
        user_tags = []
    
    #print(f'photo_id: {photo_id}, latitude: {latitude}, longitude: {longitude}, tags: {user_tags} , taken_time: {taken_time}, url: {url}, user_id: {user_id}, username: {username}, realname: {realname}, registered_location: {registered_location}, photos_count: {photos_count}')
    
    photo_list.append([photo_id, latitude, longitude, user_tags, taken_time, url, user_id, username, realname, registered_location, photos_count])

In [69]:
df = pd.DataFrame(photo_list, columns = ['photo_id', 'latitude', 'longitude', 'user_tags', 'taken_time', 'url', 'user_id', 'username', 'realname', 'registered_location', 'photos_count'])
df.head()

,photo_id,latitude,longitude,user_tags,taken_time,url,user_id,username,realname,registered_location,photos_count
0,52572081095,43.079358,-89.424034,[],2022-12-18 13:08:58,https://www.flickr.com/photos/167553772@N02/52...,167553772@N02,hienvv,Hien,Vietnam,321
1,52571909094,43.080283,-89.425107,[],2022-12-18 13:00:50,https://www.flickr.com/photos/167553772@N02/52...,167553772@N02,hienvv,Hien,Vietnam,321
2,52572080620,43.078967,-89.425096,[],2022-12-18 12:57:54,https://www.flickr.com/photos/167553772@N02/52...,167553772@N02,hienvv,Hien,Vietnam,321
3,52533098786,43.078497,-89.410344,"[retired, lakemendota, madison, nikond200, wis...",2008-08-06 16:43:10,https://www.flickr.com/photos/jcdoss/52533098786/,132625184@N08,Jason Doss,Jason Doss,Ingelheim am Rhein,9594
4,52529277687,43.087593,-89.432069,"[hawk, fall, leaves, rodent, redtailedhawk, ra...",2022-10-12 22:52:54,https://www.flickr.com/photos/mkaehrle/5252927...,193483104@N04,Martin Kaehrle,,None,75


In [70]:
df.to_csv('picnic_point_flickr_photos_info.csv', index = False)

In [34]:
# get specific attribute
photo_id = photo_info_l["photo"]["id"]
print("photo id: ", photo_id)

photo id:  52529761565


In [15]:
import time
from datetime import datetime

print("Start", datetime.now().strftime("%H:%M:%S")) # record the start time
time.sleep(3) # sleep for three second
print("End", datetime.now().strftime("%H:%M:%S")) # record the end time

Start 20:51:59
End 20:52:02
